In [16]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/SSD/Code/TinyMPC/julia/bicycle_tvlqr/Project.toml`


In [5]:
Pkg.add("ControlSystems")

   Resolving package versions...
  No Changes to `~/SSD/Code/TinyMPC/julia/bicycle_tvlqr/Project.toml`
  No Changes to `~/SSD/Code/TinyMPC/julia/bicycle_tvlqr/Manifest.toml`


In [17]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [13]:
# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

function rk4(model, ode, x, u, dt)
    k1 = dt*ode(model, x, u)
    k2 = dt*ode(model, x + k1/2, u)
    k3 = dt*ode(model, x + k2/2, u)
    k4 = dt*ode(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function get_jacobians(model, dynamics, X̄, Ū , dt)
    N_mpc = length(X̄)
    A = [FD.jacobian(_x -> rk4(model, dynamics, _x, Ū[i], dt), X̄[i]) for i = 1:(N_mpc - 1)]
    B = [FD.jacobian(_u -> rk4(model, dynamics, X̄[i], _u, dt), Ū[i]) for i = 1:(N_mpc - 1)]
    return A, B
end

get_jacobians (generic function with 1 method)

In [18]:
# ====================
# Explicit model params (working)
# ====================
@variables x[1:5] u[1:2] 
x = collect(x); u = collect(u)
x0 = [    19.4 ;
    19.5 ;
   0.676 ;
    1.51 ;
   0.175 ]; u0 = [   -1.35 ;
 -0.0615 ]
# Second-order Bicycle Dynamics
function dynamics(x, u)
    a_in = u[1]  # longitudinal acceleration 
    ϕ_in = u[2]  # steering angle rate    

    θ = x[3]  # yaw
    v = x[4]  # steering angle
    δ = x[5]

    c = cos(θ)
    s = sin(θ)

    ẋ = v*c
    ẏ = v*s
    θ̇ = v*tan(δ)
    v̇ = a_in
    δ̇ = ϕ_in  # = u2
    return [ẋ, ẏ, θ̇, v̇, δ̇]
end
function rk4(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function midpoint(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end

sym_jac_A = Symbolics.jacobian(rk4(x, u), x)
sym_jac_B = Symbolics.jacobian(rk4(x, u), u)
# Sub value to validate with ForwardDiff
sub_dict = Dict()
[sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
val_jac_A = substitute.(sym_jac_A, (sub_dict,))
val_jac_B = substitute.(sym_jac_B, (sub_dict,))
fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0)
display(norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B))
display(val_jac_A)
display(val_jac_B)

2.116336295714735e-17

5×5 Matrix{Num}:
 1  0  -0.0916626  0.0764067  -0.00684773
 0  1   0.111377   0.0645138   0.00824991
 0  0   1          0.017364    0.148602
 0  0   0          1           0
 0  0   0          0           1

5×2 Matrix{Num}:
 0.00382059   -0.000223328
 0.00322541    0.000268049
 0.000862919   0.00731157
 0.1           0
 0             0.1

In [8]:
# ====================
# Generate C code for jacobians
# ====================
fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianA_Raw, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianB_Raw, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end

In [9]:
# ====================
# Generate C code for dynamics
# ====================
dynamics_rk4 = rk4(x ,u)
dynamics_rk4 = build_function(dynamics_rk4, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Dynamics_RK4_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics_rk4.txt","a") do io
        println(io, dynamics_rk4)
end

In [ ]:
# ====================
# Symbolic model params
# not working 
# ====================
@variables g, m, ℓ, J
model = (g=g, m=m, ℓ=ℓ, J=J)
x = collect(x)
u = collect(u)
@variables x[1:6] u[1:2] 

# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end
function rk4(model, x, u)
    dt = 0.1
    k1 = dt*dynamics(model, x, u)
    k2 = dt*dynamics(model, x + k1/2, u)
    k3 = dt*dynamics(model, x + k2/2, u)
    k4 = dt*dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4) 
end
# sym_jac_A = Symbolics.jacobian(dynamics(model, x, u), x)
# sym_jac_B = Symbolics.jacobian(dynamics(model, x, u), u)
sym_jac_A = Symbolics.jacobian(rk4(model, x, u), u)

In [14]:
function lqr_controller(t, x)
    #LQR Hover Controller
    A = FD.jacobian(_x->rk4(_x, u0), xref)
    B = FD.jacobian(_u->rk4(x0, _u), uref)
    K = dlqr(A,B,Q,R)
    return u_hover - K*(x-xref)
end

lqr_controller (generic function with 1 method)

In [15]:
function closed_loop(x0,controller,N)
    xhist = zeros(length(x0),N)
    u0 = controller(1,x0)
    uhist = zeros(length(u0),N-1)
    uhist[:,1] .= u0
    xhist[:,1] .= x0
    for k = 1:(N-1)
        uk = controller(k,xhist[:,k], xref[k], uref[k])
        uhist[:,k] = max.(min.(umax, uk), umin) #enforce control limits
        xhist[:,k+1] .= rk4(xhist[:,k],uhist[:,k])
    end
    return xhist, uhist
end

closed_loop (generic function with 1 method)

In [18]:
dt = 0.1  
h = dt
Nx = 5     # number of state
Nu = 2     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
    
# Thrust limits
umax = [2; 0.5]
umin = -umax

# Cost weights
Q = Array(1.0*I(Nx));
R = Array(.01*I(Nu));
Qn = Array(1.0*I(Nx));

x_ref = [2.0; 0.0; 0; 0; 0]
x0 = [20.0; 20.0; pi/4; 0; 0]
xhist1, uhist1 = closed_loop(x0, lqr_controller, Nt);
# xhist2, uhist2 = closed_loop(x0, (t,x)->mpc_controller(t,x,x_ref), Nt);   

rk4(model, dynamics, x_hover, u_hover, dt) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0